In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt

In [5]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1


In [6]:
# Create a autograph pre-processing function to resize and normalize an image
@tf.function
def map_fn(img,label):
    image_height = 200
    image_width = 200
    # resize the image
    img = tf.image.resize(img,[image_height, image_width])
    # normalize the image
    img /= 255
    return img, label

# Prepare the dataset

Load the cat and dog dataset, splitting 80% for the training set and 20% for the test set.

In [7]:
base_dir = 'data_train/train/'

BATCH_SIZE_PER_REPLICA = 32
# Use for Mirrored Strategy
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync



train_datagen = image_dataset_from_directory(base_dir,
                                                  image_size=(200,200),
                                                  subset='training',
                                                  seed = 1,
                                                 validation_split=0.1,
                                                batch_size= BATCH_SIZE)

#len_train=train_datagen.cardinality().numpy()

train_datagen=train_datagen.map(map_fn)


test_datagen = image_dataset_from_directory(base_dir,
                                                  image_size=(200,200),
                                                  subset='validation',
                                                  seed = 1,
                                                 validation_split=0.1,
                                                  batch_size= BATCH_SIZE)


test_datagen=test_datagen.map(map_fn)

Found 25000 files belonging to 2 classes.
Using 22500 files for training.
Found 25000 files belonging to 2 classes.
Using 2500 files for validation.


In [11]:
with strategy.scope():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)),
      tf.keras.layers.MaxPool2D((2, 2)),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPool2D((2, 2)),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPool2D((2, 2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
    ])
#model.summary()

In [12]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [14]:
model.fit(train_datagen,epochs=3,validation_data=test_datagen)

Epoch 1/3


2023-08-16 16:37:32.877821: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [22500]
	 [[{{node Placeholder/_4}}]]
2023-08-16 16:37:32.879381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [22500]
	 [[{{node Placeholder/_0}}]]
2023-08-16 16:37:32.881684: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"


704/704 [==============================] - ETA: 0s - loss: 0.2517 - accuracy: 0.8967

2023-08-16 16:45:20.004681: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2500]
	 [[{{node Placeholder/_0}}]]
2023-08-16 16:45:20.005614: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2500]
	 [[{{node Placeholder/_4}}]]
2023-08-16 16:45:20.006852: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
in

704/704 [==============================] - 488s 693ms/step - loss: 0.2517 - accuracy: 0.8967 - val_loss: 0.5403 - val_accuracy: 0.8028
Epoch 2/3


...

704/704 [==============================] - 490s 695ms/step - loss: 0.1512 - accuracy: 0.9417 - val_loss: 0.7115 - val_accuracy: 0.7988
Epoch 3/3


2023-08-16 16:53:50.788690: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [22500]
	 [[{{node Placeholder/_4}}]]
2023-08-16 16:53:50.790042: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [22500]
	 [[{{node Placeholder/_4}}]]
2023-08-16 16:53:50.849268: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant
	 [[{{node Pla

704/704 [==============================] - ETA: 0s - loss: 0.0847 - accuracy: 0.9691

2023-08-16 17:01:43.118030: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2500]
	 [[{{node Placeholder/_0}}]]
2023-08-16 17:01:43.118697: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2500]
	 [[{{node Placeholder/_4}}]]
2023-08-16 17:01:43.170707: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant
	 [[{{node Plac

704/704 [==============================] - 493s 700ms/step - loss: 0.0847 - accuracy: 0.9691 - val_loss: 0.8343 - val_accuracy: 0.7972
